In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os


In [4]:
data= pd.read_csv('InsNova_data_2023_train.csv/InsNova_data_2023_train.csv')
data_vh= pd.read_csv('InsNova_data_2023_vh.csv/InsNova_data_2023_vh.csv')

In [5]:
data.head()

,id,veh_value,exposure,veh_body,veh_age,gender,area,agecat,engine_type,max_power,...,marital_status,e_bill,time_of_week_driven,time_driven,trm_len,credit_score,high_education_ind,clm,numclaims,claimcst0
0,1,0.77,0.444504,SEDAN,4,M,D,3,petrol,147,...,S,1,weekday,6pm - 12am,6,640.448137,1.0,0,0,0.0
1,2,4.45,0.562183,STNWG,1,M,A,3,petrol,158,...,S,1,weekday,6am - 12pm,12,683.749691,0.0,0,0,0.0
2,3,4.90,0.465244,STNWG,1,F,A,3,petrol,159,...,M,1,weekday,6pm - 12am,6,653.656117,1.0,0,0,0.0
3,4,0.48,0.271039,PANVN,4,M,A,4,petrol,80,...,S,1,weekday,12pm - 6pm,12,642.574671,0.0,0,0,0.0
4,5,0.85,0.141624,SEDAN,4,F,A,5,petrol,126,...,S,0,weekday,6am - 12pm,6,647.175035,0.0,0,0,0.0


In [6]:
data.numclaims.mean()

0.07312436447234626

In [10]:
num_claims=data['numclaims']
np.array(num_claims).var()


0.07847616168542991

In [5]:
!pip install xgboost


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

# Function to calculate the Gini coefficient
def gini(actual, pred):
    assert(len(actual) == len(pred))
    all = np.asarray(np.c_[actual, pred, np.arange(len(actual))], dtype=np.float)
    all = all[np.lexsort((all[:, 2], -1 * all[:, 1]))]
    total_losses = all[:, 0].sum()
    gini_sum = all[:, 0].cumsum().sum() / total_losses
    gini_sum -= (len(actual) + 1) / 2.
    return gini_sum / len(actual)

# Load the training dataset
file_path = 'InsNova_data_2023_train.csv/InsNova_data_2023_train.csv'  # Replace with your file path
train_data = pd.read_csv(file_path)

# Preprocessing: Encoding categorical variables and normalizing numerical variables
train_data_encoded = pd.get_dummies(train_data, columns=['veh_body', 'gender', 'area', 'engine_type', 'time_of_week_driven', 'time_driven', 'marital_status'], drop_first=True)
numerical_cols = ['veh_value', 'exposure', 'veh_age', 'agecat', 'max_power', 'driving_history_score', 'credit_score']
scaler = StandardScaler()
train_data_encoded[numerical_cols] = scaler.fit_transform(train_data_encoded[numerical_cols])

# Defining features and target for predicting numclaims
X = train_data_encoded.drop(['id', 'clm', 'numclaims', 'claimcst0', 'veh_color'], axis=1)  # Dropping non-feature columns
y = train_data_encoded['numclaims']  # Target variable

# Splitting the data into training and testing sets (80-20 split)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Training the RandomForestRegressor model
rf_model = RandomForestRegressor()
rf_model.fit(X_train, y_train)

# Predicting on the test set
y_pred = rf_model.predict(X_test)

# Calculating the Gini coefficient
gini_coefficient = gini(y_test, y_pred)
print("Gini Coefficient:", gini_coefficient)


Gini Coefficient: 0.09828732592046496


C:\Users\ryane\AppData\Local\Temp\ipykernel_12764\3248397478.py:11: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  all = np.asarray(np.c_[actual, pred, np.arange(len(actual))], dtype=np.float)


In [2]:
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE:", rmse)

RMSE: 0.2985094121503573


In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, PoissonRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

# Function to calculate the Gini coefficient
def gini(actual, pred):
    assert(len(actual) == len(pred))
    all = np.asarray(np.c_[actual, pred, np.arange(len(actual))], dtype=np.float)
    all = all[np.lexsort((all[:, 2], -1 * all[:, 1]))]
    total_losses = all[:, 0].sum()
    gini_sum = all[:, 0].cumsum().sum() / total_losses
    gini_sum -= (len(actual) + 1) / 2.
    return gini_sum / len(actual)

# Load the training dataset
file_path = 'InsNova_data_2023_train.csv/InsNova_data_2023_train.csv'  # Replace with your file path
train_data = pd.read_csv(file_path)

# Preprocessing
train_data_encoded = pd.get_dummies(train_data, columns=['veh_body', 'gender', 'area', 'engine_type', 'time_of_week_driven', 'time_driven', 'marital_status'], drop_first=True)
numerical_cols = ['veh_value', 'exposure', 'veh_age', 'agecat', 'max_power', 'driving_history_score', 'credit_score']
scaler = StandardScaler()
train_data_encoded[numerical_cols] = scaler.fit_transform(train_data_encoded[numerical_cols])

# Defining features and target
X = train_data_encoded.drop(['id', 'clm', 'numclaims', 'claimcst0', 'veh_color'], axis=1)
y = train_data_encoded['numclaims']

# Splitting the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Models to train
models = {
    "Linear Regression": LinearRegression(),
    "Poisson Regression": PoissonRegressor(),
    "Random Forest": RandomForestRegressor(),
    "Gradient Boosting": GradientBoostingRegressor(),
    "XGBoost": xgb.XGBRegressor()
}

# Training and evaluating models
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    gini_coeff = gini(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    print(f"{name} - Gini Coefficient: {gini_coeff}, RMSE: {rmse}")


C:\Users\ryane\AppData\Local\Temp\ipykernel_12764\3634245473.py:13: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  all = np.asarray(np.c_[actual, pred, np.arange(len(actual))], dtype=np.float)
C:\Users\ryane\AppData\Local\Temp\ipykernel_12764\3634245473.py:13: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  all = np.asarray(np.c_[actual, pre

Linear Regression - Gini Coefficient: 0.16139361165791366, RMSE: 0.290567057972532
Poisson Regression - Gini Coefficient: 0.15236187008986135, RMSE: 0.29338620106308716


C:\Users\ryane\AppData\Local\Temp\ipykernel_12764\3634245473.py:13: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  all = np.asarray(np.c_[actual, pred, np.arange(len(actual))], dtype=np.float)


Random Forest - Gini Coefficient: 0.10160424251176153, RMSE: 0.2990346241732971
Gradient Boosting - Gini Coefficient: 0.14791946929334543, RMSE: 0.2919603685048672
XGBoost - Gini Coefficient: 0.07016732156293462, RMSE: 0.30642331001899703


C:\Users\ryane\AppData\Local\Temp\ipykernel_12764\3634245473.py:13: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  all = np.asarray(np.c_[actual, pred, np.arange(len(actual))], dtype=np.float)
C:\Users\ryane\AppData\Local\Temp\ipykernel_12764\3634245473.py:13: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  all = np.asarray(np.c_[actual, pre

In [7]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor

# Hyperparameter grid
param_grid = {
    'n_estimators': [100, 150, 200],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 4, 5],
    'subsample': [0.8, 0.9, 1.0]
}

# Initialize the GridSearchCV
grid_search = GridSearchCV(estimator=GradientBoostingRegressor(), param_grid=param_grid, cv=3, n_jobs=-1, scoring='neg_mean_squared_error', verbose=3)

# Fit to the training data
grid_search.fit(X_train, y_train)

# Best parameters
print("Best parameters:", grid_search.best_params_)


Fitting 3 folds for each of 81 candidates, totalling 243 fits
Best parameters: {'learning_rate': 0.01, 'max_depth': 4, 'n_estimators': 200, 'subsample': 1.0}


In [8]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

# Retraining the model with the best parameters
optimal_model = GradientBoostingRegressor(learning_rate=0.01, max_depth=4, n_estimators=200, subsample=1.0)
optimal_model.fit(X_train, y_train)

# Predicting on the test set
y_pred_optimal = optimal_model.predict(X_test)

# Evaluating the model
gini_coefficient_optimal = gini(y_test, y_pred_optimal)
rmse_optimal = np.sqrt(mean_squared_error(y_test, y_pred_optimal))
print("Optimal Model - Gini Coefficient:", gini_coefficient_optimal)
print("Optimal Model - RMSE:", rmse_optimal)

# Cross-Validation
from sklearn.model_selection import cross_val_score

# Perform 5-fold cross-validation
cv_scores = cross_val_score(optimal_model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')

# The mean score and the 95% confidence interval of the score estimate
mean_cv_score = np.mean(cv_scores)
std_cv_score = np.std(cv_scores)
print("Cross-Validation - Mean MSE:", mean_cv_score, "Std Dev:", std_cv_score)


C:\Users\ryane\AppData\Local\Temp\ipykernel_12764\3634245473.py:13: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  all = np.asarray(np.c_[actual, pred, np.arange(len(actual))], dtype=np.float)


Optimal Model - Gini Coefficient: 0.1592078795294881
Optimal Model - RMSE: 0.2914194710111743
Cross-Validation - Mean MSE: -0.07538518232808807 Std Dev: 0.0025057817254967307


In [11]:
# Loading the validation dataset
validation_data_path = 'InsNova_data_2023_vh.csv/InsNova_data_2023_vh.csv'
validation_data = pd.read_csv(validation_data_path)

# Preprocessing the validation data similar to the training data
validation_data_encoded = pd.get_dummies(validation_data, columns=['veh_body', 'gender', 'area', 'engine_type', 'time_of_week_driven', 'time_driven', 'marital_status'], drop_first=True)
validation_data_encoded[numerical_cols] = scaler.transform(validation_data_encoded[numerical_cols])

# Ensure the same columns in validation data as in training data
validation_data_encoded = validation_data_encoded.reindex(columns=X_train.columns, fill_value=0)

# Predicting on the validation dataset
validation_predictions = optimal_model.predict(validation_data_encoded)

# Creating a DataFrame for submission
submission_df = pd.DataFrame({
    'id': validation_data['id'],
    'Predict': validation_predictions
})

# Checking the first few rows of the submission file
submission_df.max()


id         22620.000000
Predict        1.157499
dtype: float64

In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

# Load the dataset
file_path = 'InsNova_data_2023_train.csv/InsNova_data_2023_train.csv'  # Update the path accordingly
data = pd.read_csv(file_path)

# Separating the target variable and features
X = data.drop('numclaims', axis=1)
y = data['numclaims']

# Identifying numerical and categorical columns
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = X.select_dtypes(include=['object']).columns

# Creating transformers for numerical and categorical data
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundling preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Splitting the dataset into training and testing sets (80-20 split)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Function to train a model and get predictions
def train_and_predict(model, X_train, y_train, X_test):
    pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                               ('model', model)])
    pipeline.fit(X_train, y_train)
    predictions = pipeline.predict(X_test)
    return predictions

# Training and predicting with each model
lr_predictions = train_and_predict(LinearRegression(), X_train, y_train, X_test)
rf_predictions = train_and_predict(RandomForestRegressor(random_state=0), X_train, y_train, X_test)
gb_predictions = train_and_predict(GradientBoostingRegressor(random_state=0), X_train, y_train, X_test)

# Calculating MSE for each model
lr_mse = mean_squared_error(y_test, lr_predictions)
rf_mse = mean_squared_error(y_test, rf_predictions)
gb_mse = mean_squared_error(y_test, gb_predictions)

# Calculating weights inversely proportional to the MSE
total_inverse_mse = (1 / lr_mse) + (1 / rf_mse) + (1 / gb_mse)
lr_weight = (1 / lr_mse) / total_inverse_mse
rf_weight = (1 / rf_mse) / total_inverse_mse
gb_weight = (1 / gb_mse) / total_inverse_mse

# Combining predictions using weighted average
combined_predictions = (lr_weight * lr_predictions) + (rf_weight * rf_predictions) + (gb_weight * gb_predictions)

# Displaying combined predictions
print(combined_predictions[combined_predictions>1])


[1.0851712  1.02389089 1.0715956  1.08637129 1.02985773 1.0342072
 1.07341546 1.04825894 1.0259283  1.13714216 1.21276976 1.02216809
 1.02547954 1.1105667  1.15675799 1.17829313 1.08466942 1.0246307
 1.04011292 1.09623875 1.02651615 1.06601708 1.02731963 1.27480207
 1.03130111 1.02765691 1.07090419 1.04478271 1.02536096 1.09297915
 1.07363894 1.01895365 1.02623881 1.23257165 1.0090989  1.13993042
 1.17482325 1.17679507 1.00709813 1.0505753  1.062162   1.09881837
 1.08784966 1.72370977 1.12581923 1.06071094 1.03782956 1.13440398
 1.19000083 1.1479211  1.23164721 1.06710672 1.01819194 1.02592467
 1.03292433 1.0346937  1.02772265 1.03319592 1.03465041 1.0261923
 1.01895048 1.09246272 1.02879199 1.07332932 1.01766794 1.31675933
 1.0713949  1.13006875 1.15618688 1.08408576 1.02144095 1.09302293
 1.12067045 1.02360102 1.06769625 1.10289663 1.09713601 1.08595291
 1.11500659 1.14251035 1.08884746 1.04582977 1.02798298 1.13484575
 1.183595   1.02724615 1.13162957 1.01580575 1.13023511 1.0242539

In [16]:
print(combined_predictions[combined_predictions>1])

[1.0851712  1.02389089 1.0715956  1.08637129 1.02985773 1.0342072
 1.07341546 1.04825894 1.0259283  1.13714216 1.21276976 1.02216809
 1.02547954 1.1105667  1.15675799 1.17829313 1.08466942 1.0246307
 1.04011292 1.09623875 1.02651615 1.06601708 1.02731963 1.27480207
 1.03130111 1.02765691 1.07090419 1.04478271 1.02536096 1.09297915
 1.07363894 1.01895365 1.02623881 1.23257165 1.0090989  1.13993042
 1.17482325 1.17679507 1.00709813 1.0505753  1.062162   1.09881837
 1.08784966 1.72370977 1.12581923 1.06071094 1.03782956 1.13440398
 1.19000083 1.1479211  1.23164721 1.06710672 1.01819194 1.02592467
 1.03292433 1.0346937  1.02772265 1.03319592 1.03465041 1.0261923
 1.01895048 1.09246272 1.02879199 1.07332932 1.01766794 1.31675933
 1.0713949  1.13006875 1.15618688 1.08408576 1.02144095 1.09302293
 1.12067045 1.02360102 1.06769625 1.10289663 1.09713601 1.08595291
 1.11500659 1.14251035 1.08884746 1.04582977 1.02798298 1.13484575
 1.183595   1.02724615 1.13162957 1.01580575 1.13023511 1.0242539

In [17]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

# Load the training dataset
file_path_train = 'InsNova_data_2023_train.csv/InsNova_data_2023_train.csv'  # Update with your file path
data_train = pd.read_csv(file_path_train)

# Separating the target variable and features for the training dataset
X_train_full = data_train.drop('numclaims', axis=1)
y_train_full = data_train['numclaims']

# Identifying numerical and categorical columns from the training data
numerical_cols = X_train_full.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = X_train_full.select_dtypes(include=['object']).columns

# Creating transformers for numerical and categorical data
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundling preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Splitting the training dataset (optional if you want to evaluate models on the training data)
X_train, X_test, y_train, y_test = train_test_split(X_train_full, y_train_full, test_size=0.2, random_state=0)

# Load the validation dataset
file_path_vh = 'InsNova_data_2023_vh.csv/InsNova_data_2023_vh.csv'  # Update with your file path
data_vh = pd.read_csv(file_path_vh)

# Adjusting the preprocessing to only include common columns
numerical_cols_common = [col for col in numerical_cols if col in data_vh.columns]
categorical_cols_common = [col for col in categorical_cols if col in data_vh.columns]

# Updated preprocessor for the validation dataset
preprocessor_common = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols_common),
        ('cat', categorical_transformer, categorical_cols_common)
    ])

# Function to train a model on the training set and predict on a given dataset
def train_and_predict_common(model, X_train, y_train, X_predict):
    pipeline = Pipeline(steps=[('preprocessor', preprocessor_common),
                               ('model', model)])
    pipeline.fit(X_train, y_train)
    predictions = pipeline.predict(X_predict)
    return predictions

# Training models and predicting on the validation dataset
lr_predictions_vh = train_and_predict_common(LinearRegression(), X_train_full, y_train_full, data_vh)
rf_predictions_vh = train_and_predict_common(RandomForestRegressor(random_state=0), X_train_full, y_train_full, data_vh)
gb_predictions_vh = train_and_predict_common(GradientBoostingRegressor(random_state=0), X_train_full, y_train_full, data_vh)

# Combining predictions using weighted average
# First, compute weights based on MSE of each model on the training split (if you did this step)
# If not, you can assign equal weights or based on model confidence
# Example with equal weights: (You can modify this part based on your model evaluation)
weights = [1/3, 1/3, 1/3]  # Equal weights for LR, RF, GB
combined_predictions_vh = (weights[0] * lr_predictions_vh) + (weights[1] * rf_predictions_vh) + (weights[2] * gb_predictions_vh)

# Adding the combined predictions as a new column in the validation dataset
data_vh['num_claim'] = combined_predictions_vh

# Save the updated validation dataset with predictions
output_path = 'numberofclaims.csv'  # Update with your file path
data_vh.to_csv(output_path, index=False)


In [20]:
len(data_vh[data_vh['num_claim']>0.5])

3

In [22]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression

# Function to calculate the Gini index for regression
def gini(actual, pred):
    assert(len(actual) == len(pred))
    all = np.asarray(np.c_[actual, pred, np.arange(len(actual))], dtype=np.float)
    all = all[np.lexsort((all[:, 2], -1*all[:, 1]))]
    total_losses = all[:, 0].sum()
    gini_sum = all[:, 0].cumsum().sum() / total_losses
    gini_sum -= (len(actual) + 1) / 2.0
    return gini_sum / len(actual)

def normalized_gini(actual, pred):
    return gini(actual, pred) / gini(actual, actual)

# Load the data
train_data = pd.read_csv('InsNova_data_2023_train.csv/InsNova_data_2023_train.csv')
vh_data = pd.read_csv('InsNova_data_2023_vh.csv/InsNova_data_2023_vh.csv')

# Preprocessing steps
categorical_cols = train_data.select_dtypes(include=['object', 'category']).columns
numerical_cols = train_data.select_dtypes(include=['int64', 'float64']).columns
numerical_cols = numerical_cols.drop(['clm', 'numclaims', 'claimcst0'])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', numerical_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ])

# Splitting the data
X_train, X_valid, y_train, y_valid = train_test_split(
    train_data.drop(['clm', 'numclaims', 'claimcst0'], axis=1), 
    train_data['claimcst0'], 
    test_size=0.2, 
    random_state=0
)

# Initialize models
models = {
    "RandomForestRegressor": RandomForestRegressor(n_estimators=100, random_state=0),
    "GradientBoostingRegressor": GradientBoostingRegressor(n_estimators=100, random_state=0),
    #"LinearRegression": LinearRegression()
}

# Train and evaluate models
results = {}
for model_name, model in models.items():
    pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('model', model)])
    pipeline.fit(X_train, y_train)
    valid_preds = pipeline.predict(X_valid)
    mse = mean_squared_error(y_valid, valid_preds)
    gini_score = normalized_gini(y_valid, valid_preds)
    results[model_name] = {'MSE': mse, 'Gini': gini_score}

# Select the best model
best_model_name = min(results, key=lambda k: (results[k]['MSE'], -results[k]['Gini']))
best_model = models[best_model_name]

# Train best model on the entire training set
best_pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('model', best_model)])
best_pipeline.fit(train_data.drop(['clm', 'numclaims', 'claimcst0'], axis=1), train_data['claimcst0'])

# Predict with the best model
vh_preds_best = best_pipeline.predict(vh_data)

# Prepare output
output_best = pd.DataFrame({'id': vh_data['id'], 'Predict': vh_preds_best})

# Save output to CSV
output_best.to_csv('predictions_2.csv', index=False)


C:\Users\ryane\AppData\Local\Temp\ipykernel_12764\2532396906.py:14: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  all = np.asarray(np.c_[actual, pred, np.arange(len(actual))], dtype=np.float)
C:\Users\ryane\AppData\Local\Temp\ipykernel_12764\2532396906.py:14: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  all = np.asarray(np.c_[actual, pre

In [42]:
import pandas as pd
import numpy as np
from statsmodels.formula.api import glm
import statsmodels.api as sm

# Load the training data
train_data = pd.read_csv('InsNova_data_2023_train.csv/InsNova_data_2023_train.csv')

# Check mean and variance of numclaims
mean_numclaims = train_data['numclaims'].mean()
var_numclaims = train_data['numclaims'].var()

print(f"Mean of numclaims: {mean_numclaims}")
print(f"Variance of numclaims: {var_numclaims}")

# Poisson regression model for Frequency
#formula = 'numclaims ~ np.log(exposure) + agecat + area + veh_value + veh_age + veh_value:veh_age + area:veh_value'
#frequency_model = glm(formula, data=train_data, family=sm.families.Poisson()).fit()

formula = 'numclaims ~  exposure + agecat + area + veh_value + veh_age + gender + driving_history_score  + max_power + credit_score '
frequency_model = glm(formula, data=train_data).fit()

print(frequency_model.summary())


Mean of numclaims: 0.07312436447234626
Variance of numclaims: 0.0784796313185289
                 Generalized Linear Model Regression Results                  
Dep. Variable:              numclaims   No. Observations:                22619
Model:                            GLM   Df Residuals:                    22605
Model Family:                Gaussian   Df Model:                           13
Link Function:               identity   Scale:                        0.077320
Method:                          IRLS   Log-Likelihood:                -3137.9
Date:                Thu, 16 Nov 2023   Deviance:                       1747.8
Time:                        05:05:25   Pearson chi2:                 1.75e+03
No. Iterations:                     3   Pseudo R-squ. (CS):            0.01545
Covariance Type:            nonrobust                                         
                            coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------

In [43]:
train_data.describe()

,id,veh_value,exposure,veh_age,agecat,max_power,driving_history_score,e_bill,trm_len,credit_score,high_education_ind,clm,numclaims,claimcst0
count,22619.000000,22619.000000,22619.000000,22619.000000,22619.000000,22619.000000,22619.000000,22619.00000,22619.000000,22619.000000,22619.000000,22619.000000,22619.000000,22619.000000
mean,11310.000000,1.868948,0.433038,2.667492,3.481675,152.447765,70.868341,0.63681,10.493833,649.875077,0.117114,0.068173,0.073124,163.048084
std,6529.687205,1.278588,0.272899,1.070555,1.420252,51.834156,19.083318,0.48093,2.601685,10.515364,0.321563,0.252048,0.280142,1271.955238
min,1.000000,0.000000,0.001754,1.000000,1.000000,60.000000,1.000000,0.00000,6.000000,607.797435,0.000000,0.000000,0.000000,0.000000
25%,5655.500000,1.070000,0.203696,2.000000,2.000000,115.000000,58.000000,0.00000,6.000000,643.134748,0.000000,0.000000,0.000000,0.000000
50%,11310.000000,1.570000,0.384313,3.000000,3.000000,144.000000,72.000000,1.00000,12.000000,647.180197,0.000000,0.000000,0.000000,0.000000
75%,16964.500000,2.260000,0.643316,4.000000,5.000000,180.000000,86.000000,1.00000,12.000000,654.069205,0.000000,0.000000,0.000000,0.000000
max,22619.000000,24.510000,0.999378,4.000000,6.000000,409.000000,99.000000,1.00000,12.000000,790.360253,1.000000,1.000000,3.000000,57895.584560


In [44]:
# Inverse Gaussian model for Severity
# Create a new column for the average claim cost
train_data_with_claims['avg_claim_cost'] = train_data_with_claims['claimcst0'] / train_data_with_claims['numclaims']

# Inverse Gaussian model for Severity
#severity_model = glm('avg_claim_cost ~ gender + veh_age + agecat', 
 #                    data=train_data_with_claims, 
  #                   family=sm.families.InverseGaussian(link=sm.families.links.log())).fit()

severity_model = glm('avg_claim_cost ~ gender + veh_age + agecat + max_power + veh_value + driving_history_score + exposure + credit_score', 
                     data=train_data_with_claims).fit()

print(severity_model.summary())
""


                 Generalized Linear Model Regression Results                  
Dep. Variable:         avg_claim_cost   No. Observations:                 1542
Model:                            GLM   Df Residuals:                     1533
Model Family:                Gaussian   Df Model:                            8
Link Function:               identity   Scale:                      1.7236e+07
Method:                          IRLS   Log-Likelihood:                -15030.
Date:                Thu, 16 Nov 2023   Deviance:                   2.6423e+10
Time:                        05:05:27   Pearson chi2:                 2.64e+10
No. Iterations:                     3   Pseudo R-squ. (CS):            0.01544
Covariance Type:            nonrobust                                         
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept              5192.66

''

In [45]:
from sklearn.model_selection import KFold
import numpy as np

def gini(actual, pred):
    assert len(actual) == len(pred)
    all = np.asarray(np.c_[actual, pred, np.arange(len(actual))], dtype=np.float)
    all = all[np.lexsort((all[:, 2], -1 * all[:, 1]))]
    total_losses = all[:, 0].sum()
    gini_sum = all[:, 0].cumsum().sum() / total_losses
    gini_sum -= (len(actual) + 1) / 2.
    return gini_sum / len(actual)

def normalized_gini(actual, pred):
    return gini(actual, pred) / gini(actual, actual)

# Prepare for cross-validation
kf = KFold(n_splits=10, shuffle=True, random_state=1)
gini_scores = []

for train_index, test_index in kf.split(train_data):
    cv_train, cv_test = train_data.iloc[train_index], train_data.iloc[test_index]
    
    # Fit the model on cv_train
    #cv_model = glm(formula, data=cv_train, family=sm.families.Poisson()).fit()
    cv_model = glm(formula, data=cv_train).fit()
    # Predict on cv_test
    predictions = cv_model.predict(cv_test)
    
    # Compute and store Gini coefficient
    gini_score = normalized_gini(cv_test['numclaims'], predictions)
    gini_scores.append(gini_score)

# Calculate the average Gini coefficient
average_gini = np.mean(gini_scores)
print(f"Average Gini Coefficient from Cross-Validation: {average_gini}")


C:\Users\ryane\AppData\Local\Temp\ipykernel_14984\1342160008.py:6: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  all = np.asarray(np.c_[actual, pred, np.arange(len(actual))], dtype=np.float)
C:\Users\ryane\AppData\Local\Temp\ipykernel_14984\1342160008.py:6: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  all = np.asarray(np.c_[actual, pred,

Average Gini Coefficient from Cross-Validation: 0.2860533099866334


C:\Users\ryane\AppData\Local\Temp\ipykernel_14984\1342160008.py:6: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  all = np.asarray(np.c_[actual, pred, np.arange(len(actual))], dtype=np.float)


In [46]:
print(f"Average Gini Coefficient from Cross-Validation: {average_gini}")

Average Gini Coefficient from Cross-Validation: 0.2860533099866334


In [47]:
import random

def bootstrap_model(data, model_formula, family, R=500):
    boot_coefs = []

    for _ in range(R):
        boot_sample = data.sample(frac=1, replace=True, random_state=random.randint(1, 10000))
        boot_model = glm(model_formula, data=boot_sample, family=family).fit()
        boot_coefs.append(boot_model.params)

    # Convert list of coefficients to DataFrame
    boot_coefs_df = pd.DataFrame(boot_coefs)

    # Calculate the average coefficients
    avg_coefs = boot_coefs_df.mean(axis=0)
    return avg_coefs

# Apply bootstrapping
avg_boot_coefs = bootstrap_model(train_data, formula, sm.families.Poisson(), R=500)
print(avg_boot_coefs)


Intercept               -2.279430
area[T.B]               -0.030603
area[T.C]               -0.003049
area[T.D]               -0.158172
area[T.E]               -0.145635
area[T.F]                0.031815
gender[T.M]             -0.078984
exposure                 1.564745
agecat                  -0.086688
veh_value                0.009627
veh_age                  0.010251
driving_history_score    0.001758
max_power                0.000023
credit_score            -0.001424
dtype: float64


In [48]:
# Load the vh data
vh_data = pd.read_csv('InsNova_data_2023_vh.csv/InsNova_data_2023_vh.csv')
# Compute Frequency and Severity predictions
# (Replace `frequency_model` and `severity_model` with your actual model objects)
vh_data['predicted_frequency'] = frequency_model.predict(vh_data)
vh_data['predicted_severity'] = severity_model.predict(vh_data)

# Calculate the final claim cost predictions
vh_data['predicted_claimcst0'] = vh_data['predicted_frequency'] * vh_data['predicted_severity']
# Prepare the output DataFrame
output = vh_data[['id', 'predicted_claimcst0']]
output.rename(columns={'predicted_claimcst0': 'Predict'}, inplace=True)
# Export to CSV
output.to_csv('final_predictions2.csv', index=False)



C:\Users\ryane\AppData\Local\Temp\ipykernel_14984\3911342461.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  output.rename(columns={'predicted_claimcst0': 'Predict'}, inplace=True)


In [51]:
output[output['Predict'] <0]['Predict']=0

C:\Users\ryane\AppData\Local\Temp\ipykernel_14984\1527340432.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  output[output['Predict'] <0]['Predict']=0


In [55]:
# Assigning 0 to negative values of Predict
output.loc[output['Predict'] < 0, 'Predict'] = 0
output.to_csv('final_predictions2.csv', index=False)